## Proceso de ETL.

Carga del dataset


In [2]:
import pandas as pd

# Cargar dataset
df = pd.read_csv("customer_shopping_data.csv")

df


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


## TRANSFORMACIÓN

En esta fase, aislamos los atributos descriptivos del conjunto de datos principales para crear tablas de dimensiones. Cada tabla representará una entidad de negocio única (Cliente, Producto, Fecha, Ubicación y Método de Pago).


El método .drop_duplicates() es fundamental aquí, ya que garantiza que cada tabla de dimensión contenga solo valores únicos, eliminando la redundancia.


In [ ]:
# Crear dataframes únicos para cada dimensión
dim_customer = df[['customer_id', 'gender', 'age']].drop_duplicates().reset_index(drop=True)
dim_product = df[['category']].drop_duplicates().reset_index(drop=True)
dim_date = df[['invoice_date']].drop_duplicates().reset_index(drop=True)
dim_location = df[['shopping_mall']].drop_duplicates().reset_index(drop=True)
dim_payment = df[['payment_method']].drop_duplicates().reset_index(drop=True)

# Agregar Claves Sustitutas
dim_customer['customer_key'] = dim_customer.index + 1
dim_product['product_key'] = dim_product.index + 1
dim_date['date_key'] = dim_date.index + 1
dim_location['location_key'] = dim_location.index + 1
dim_payment['payment_key'] = dim_payment.index + 1

# Merge para construir la tabla de hechos
fact_sales = df.merge(dim_customer, on=['customer_id', 'gender', 'age'])
fact_sales = fact_sales.merge(dim_product, on='category')
fact_sales = fact_sales.merge(dim_date, on='invoice_date')
fact_sales = fact_sales.merge(dim_location, on='shopping_mall')
fact_sales = fact_sales.merge(dim_payment, on='payment_method')

fact_sales = fact_sales[['invoice_no',
                         'customer_key',
                         'product_key',
                         'date_key',
                         'location_key',
                         'payment_key',
                         'quantity',
                         'price']]

# ==============

# CONEXIÓN A POSTGRESQL CON SQLACHEMY

In [4]:
from sqlalchemy import create_engine

# Datos de conexión
user = "mi_usuario"
password = "1122919783"
host = "localhost"
port = "5432"
database = "ProyectoCD"

# Cadena de conexión SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")


# CARGAR DIMENSIONES
Esta es la fase de Carga del ETL. Una vez que los datos están transformados y estructurados en nuestro modelo de estrella, el siguiente paso es cargarlos en la bodega de datos en PostgreSQL.

In [5]:
# Cargar dimensiones
dim_customer[['customer_key', 'customer_id', 'gender', 'age']].to_sql(
    "dim_customer", engine, if_exists='append', index=False)

dim_product[['product_key', 'category']].to_sql(
    "dim_product", engine, if_exists='append', index=False)

dim_date[['date_key', 'invoice_date']].to_sql(
    "dim_date", engine, if_exists='append', index=False)

dim_location[['location_key', 'shopping_mall']].to_sql(
    "dim_location", engine, if_exists='append', index=False)

dim_payment[['payment_key', 'payment_method']].to_sql(
    "dim_payment", engine, if_exists='append', index=False)

3

CARGAR HECHOS

In [6]:
# Cargar hechos
fact_sales.to_sql("fact_sales", engine, if_exists='append', index=False)

print("✅ Proceso ETL finalizado. Datos cargados en PostgreSQL")

✅ Proceso ETL finalizado. Datos cargados en PostgreSQL
